# ENPH 213 - Week 5 Lab - Part 6

In this lab, we will be working on solving non-linear equations using a variety of methods while continuing to develop your Python skills.

When you are finished, please rename this notebook to LastName_ENPH213_Lab5-Part6, where LastName is your last name.  Submit that file to onQ along with the notebook for Parts 1-5. 

For marking Parts 1-4 will be marked together (Weighted out of 10) and Parts 5 and 6 will be marked together (Weighted out of 5).

In [1]:
#Importing modules
import math as m
import numpy as np
import cmath as cm
from matplotlib import pyplot as plt
%matplotlib inline

# Part 6

Find the roots to following set of functions

$\Large f_1(x,y,z) = 2x - y \cos z - 3 \\
\Large f_2(x,y,z) = x^2 - 25(y - 2)^2 + \sin z - \frac{\pi}{10} \\
\Large f_3(x,y,z) = 7xe^y - 17z + 8\pi$

To solve this system of functions, use the generalized Newton's Method as discussed in class.  Use the Jacobian Matrix, and construct it by analytically/symbollically calculating the partial derivatives for its elements.  Iterate the solutions of the matrix equations by adding the evaluated steps back into the previous guess values for $x$, $y$, and $z$.  For this question, you may fulfill the iteration sensitivity when either the total of the step components or the total of the residual function values acheives an error less than 0.00001.

Provide the determine roots of the set of functions as well as the number of iterations needed.

**Note:** You will need the $BackSub(A, b)$ and related functions from Lab 4 to perform the matrix calculations.  To do this, from your Lab 4 Jupyter notebook, select "File-->Download as-->Python (.py)".  Save this file as "Lab4.py" in the directory of this notebook.  Then below, add the line "from Lab4 import BackSub".  It may load and perform all computations, but you will not have access to your previous functions in this notebook.  If you click on the margin beside this cell, it will reduce in size.  If you double click, it should completely collapse.

I decided it would be easier to copy and paste the functions from LAB 4 and test them, rather than downloading the lab file.

In [2]:
def GaussPivot(Ab , col):
    #goal is to simulate the rows that need to be pivoted
    if ((np.max(Ab[col:,col])) > (Ab[col,col])): # test codition for swapping rows
       
        Ab[[col,np.argmax(Ab[col:,col]) + col],:] = Ab[[np.argmax(Ab[col:,col]) + col,col], :] #swapping rows
        
    return Ab

In [3]:
def GaussElim(Ab,col):
    #call the pivot func
    Ab1 = GaussPivot(Ab, col)
    Nrows = Ab.shape[0]  # set the bounds of the loop to the rows in the array
    
    # loop through the array and use the eqn given
    # row i = row i - Aij/Ajj * row j
    for i in range(col+1, Nrows):
       
        Ab1[i][:] = Ab1[i][:]-(Ab1[i][col]/(np.copy(Ab1[col][col])))*Ab1[col][:]
    
    return Ab1

In [4]:
def UpTriang(A, b):
    
    column = A.shape[0] # number of col
    
    Ab = np.concatenate([np.array(A),np.transpose(np.array(b))], 1) # making the new array with b on A
  
    copy = np.copy(Ab) # copying the array
    
    for i in range(column): # looping through the array
        copy = GaussElim(copy,i) # calling our elim func for increasing columns
        
    return copy

In [5]:
#Function from Lab 4
def BackSub(A,b):
    
    upperA = UpTriang(A,b) # call the Uptriangle Func
    
    for i in range(A.shape[0]): # loop through for num columns
        
        upperA[i,:] = upperA[i,:]/upperA[i,i]
        
    copy = np.copy(upperA) # make a copy of array
    
    # nest for loops to iterate throught the array
    for a in range(upperA.shape[0]-1,-1,-1):
        
        for b in range(a-1,-1,-1):
            
            copy[b,:] = copy[b,:]-upperA[a,:]*upperA[b,a] # subtract and set new value
            
            upperA = np.copy(copy)
            
            newArray = upperA[:,A.shape[0]] 
    return newArray

In [6]:
# **** Code to test the implementation of the functions written in LAB 4*************

A = np.array([[10, 9, 8, 7],[1, 2, 3, 4], [6, 6, 3, 2], [1, 5, 4, 7]])
b = np.array([[6, 5, 1, 9]])
A = A.astype(float)
b = b.astype(float)

testArray = BackSub(A,b) # check the function by calling it with A and b
print("The Solution for the system of eqns is\n",testArray) # print the result
sizeB = np.size(b) # find the size of array b to iterate through
check = np.ones(sizeB) # initalize the check array


sumb = np.sum(b[0,:]) # taking the sum of b
#print(sumb)

sumA = np.sum(testArray*A[0:sizeB,:]) # multiplying the result by A, for N rows
#print(sumA)

# now need to check if there is an actual solution
check = sumb - sumA
check = check.astype(int)
#print(check)

# Conditional prints
if (check == 0): 
    print("\nThe Solution is Correct, The Code From LAB 4 has been corrected implemented")
else:
    print("\nNot a True Soln")

The Solution for the system of eqns is
 [-1.   0.5  1.   0.5]

The Solution is Correct, The Code From LAB 4 has been corrected implemented


In [7]:
# **NEW CODE FOR LAB 5 PART 6**

## Need  to update variable names ect
error = 0.00001
# the three functions given in the question
def f1 (x):
    f = 2*x[0] - x[1]*np.cos(x[2]) - 3
    return f

def f2 (x):    
    f = x[0]**2 - 25*(x[1]-2)**2 + np.sin(x[2]) - np.pi/10
    return f

def f3 (x):    
    f = 7*x[0]*np.exp(x[1]) - 17*x[2] + 8*np.pi
    return f


#function to make an array of the three functions to pass to BackSub
def fArray(x):     
    farray = -1*np.reshape([f1(x), f2(x), f3(x)],(1,3))
    return farray


# funtion that takes the partial derivatives of the functions in the array
def df (f, x):
    step = 0.01 # define the step size
    length = len(x) # store the length of x
    
    identityM = np.zeros((length,length))     # identity Matrix
    dervArray = np.zeros((length,length))  # array to store deriv
    
    for i in range(length): # set diagonals equal to step
        identityM[i,i] = step
        
    for j in range(length):       # put the values into derv array
        dervArray[j] = np.array((f(x - identityM[:,j]) - f(x))/step)
    
    array = np.transpose(dervArray) # fix the position of values
    
    return array

# Test print for the given functions
print (" Test array\n",df(fArray, [1,1,1]))

 Test array
 [[  2.          -0.54030231   0.83875547]
 [  1.99        50.25         0.54450062]
 [ 19.0279728   18.93314928 -17.        ]]


In [8]:
# function that takes the function array and a guess of root location and returns roots within error
def Newton(f, roots):
    i = 0 #create counter
    while (np.sum(abs(f(roots))) > error): #while not within the error
        
        #step for each guess is passed to backsub
        step = BackSub(df(f, roots), f(roots))    
        roots = roots + step   # add the step to the guess
        i = i+1 #increment counter
        
    print("\nTook ", i,"iterations to find the root") 
    return roots           #returns the roots                 

solution = Newton(fArray, [1,1,1])      # call the Newton function
print("The calculated roots within",error,"error of the true roots are: ",solution)


Took  9 iterations to find the root
The calculated roots within 1e-05 error of the true roots are:  [0.53897753 1.93684326 3.01789797]


## Acknowledgements

Please comment on any help that you received from your group members or others concerning this Lab assignment.

All code was written by Nathan Pacey, however topics were discussed Connor Legg and Stuart Gaherty